# Knowledge Graph → AAS JSON Conversion Pipeline

This notebook demonstrates the transformation of:
- an **OWL-based Knowledge Graph (KG)**  
- via a **SPARQL CONSTRUCT mapping**  
- into **ISO 63278 Asset Administration Shell (AAS) JSON**  

Pipeline Steps:

1. Load KG (OWL/Turtle) from DSMS instance
2. Apply SPARQL mapping → AAS RDF
3. Convert AAS RDF → AAS JSON (using `py-aas-rdf`)
4. Validate JSON against AAS Meta-Model schema
5. Save output JSON


## Imports & Environment Setup

In [1]:
from dsms import DSMS
import requests
from rdflib import RDF, Graph, Namespace
import os
import json
from jsonschema import validate
from py_aas_rdf.models.asset_administraion_shell import AssetAdministrationShell
from py_aas_rdf.models.concept_description import ConceptDescription
from py_aas_rdf.models.submodel import Submodel
from pyoxigraph import RdfFormat, Store

## Variable initialization and load configuration from .env to initialize DSMS instance

In [2]:
AAS = Namespace("https://admin-shell.io/aas/3/0/")
PLACEHOLDER = "https://example.org"


AAS_SCHEMA_URL = (
    "https://raw.githubusercontent.com/admin-shell-io/aas-specs/refs/heads/master/schemas/json/aas.json"
)

# User paths
MAPPING_PATH = "./input/mapping.sparql"
OUTPUT_DIR = "./output"

os.makedirs(OUTPUT_DIR, exist_ok=True)

HOST_URL = "https://pmdx.materials-data.space/"
KITEM_ID = "8de5e175-ea5e-40c9-9ba9-62649720f898"
UNIQUE_ID = HOST_URL + KITEM_ID

# DSMS setup
dsms = DSMS(env=".env")


/home/kiran/work/mapro/KG2AAS-prototype/venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pmdx.materials-data.space'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/kiran/work/mapro/KG2AAS-prototype/venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pmdx.materials-data.space'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/kiran/work/mapro/KG2AAS-prototype/venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pmdx.materials-data.space'. Adding certificate verification is strongly advised. See: https://urllib3.readthedoc

## Apply KG2AAS pipeline

In [3]:
# ---------------------------------------------------------
# STEP 1 — Fetch the Knowledge Graph in TTL
# ---------------------------------------------------------
export_url = f"{HOST_URL}/api/knowledge/contexts/{KITEM_ID}/export"
response = requests.get(export_url)
ttl_data = response.text
print("Fetched TTL successfully.")

# ---------------------------------------------------------
# STEP 2 — Load SPARQL CONSTRUCT Mapping
# ---------------------------------------------------------
with open(MAPPING_PATH, "r", encoding="utf-8") as f:
    mapping_query = f.read().replace(PLACEHOLDER, UNIQUE_ID)

# ---------------------------------------------------------
# STEP 3 — Apply Mapping with OxiGraph
# ---------------------------------------------------------
store = Store()
store.load(ttl_data, format=RdfFormat.TURTLE)
mapping_result = store.query(mapping_query)

mapped_ttl = mapping_result.serialize(format=RdfFormat.TURTLE)
print("Applied SPARQL mapping.")

# ---------------------------------------------------------
# STEP 4 — Convert Result to RDFLib Graph
# ---------------------------------------------------------
g = Graph().parse(data=mapped_ttl, format="turtle")

# ---------------------------------------------------------
# STEP 5 — Extract AAS RDF Entities
# ---------------------------------------------------------
aas_shells = list(g.subjects(RDF.type, AAS["AssetAdministrationShell"]))
submodels = list(g.subjects(RDF.type, AAS["Submodel"]))
cds = list(g.subjects(RDF.type, AAS["ConceptDescription"]))

print(f"Found {len(aas_shells)} shells, {len(submodels)} submodels, {len(cds)} CDs")

# ---------------------------------------------------------
# STEP 6 — Transform AAS RDF → AAS JSON
# ---------------------------------------------------------
aas_json = {
    "assetAdministrationShells": [
        AssetAdministrationShell.from_rdf(g, s).model_dump(exclude_none=True, mode="json")
        for s in aas_shells
    ],
    "submodels": [
        Submodel.from_rdf(g, s).model_dump(exclude_none=True, mode="json")
        for s in submodels
    ],
    "conceptDescriptions": [
        ConceptDescription.from_rdf(g, s).model_dump(exclude_none=True, mode="json")
        for s in cds
    ],
}

# ---------------------------------------------------------
# STEP 7 — Validate Against AAS JSON Schema
# ---------------------------------------------------------
schema = requests.get(AAS_SCHEMA_URL).json()
validate(aas_json, schema)
print("AAS JSON is schema-valid.")

# ---------------------------------------------------------
# STEP 8 — Save AAS JSON
# ---------------------------------------------------------
output_file = os.path.join(OUTPUT_DIR, f"{KITEM_ID}_AAS.json")
with open(output_file, "w") as f:
    f.write(json.dumps(aas_json, indent=2))

print(f"✓ Saved: {output_file}")


Fetched TTL successfully.
Applied SPARQL mapping.
Found 1 shells, 1 submodels, 71 CDs
AAS JSON is schema-valid.
✓ Saved: ./output/8de5e175-ea5e-40c9-9ba9-62649720f898_AAS.json
